In [ ]:
pip install tensorflow

In [24]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
import tensorflow.keras.optimizers 
from tensorflow.keras.models import Model

(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
tf.random.set_seed(42)

In [ ]:
Y_train ## what to normalize (uint8, 0,254 (255 values))

In [25]:
X_train = X_train[0:2000] 
Y_train = Y_train[0:2000] 
X_test = X_test[0:500]
Y_test = Y_test[0:500]

## Normalize values
X_train, X_test = X_train / 255.0, X_test / 255.0 

## Load model
base_model = MobileNetV2(weights='imagenet', include_top=False)

## Freeze base layers
base_model.trainable = False

## Add classifiers
shape = tf.keras.Input(shape=(32,32,3)) ## there was no specified input shape, so i am reusing from the similar task
model_classifier = base_model(inputs=shape,training=False)
model_classifier = GlobalAveragePooling2D()(model_classifier)
model_classifier = Dense(units=128, activation='relu')(model_classifier)
model_classifier = Dropout(rate=0.02)(model_classifier)
outputs = Dense(units=10, activation='softmax')(model_classifier)

model = Model(inputs=shape, outputs=outputs)

adam_optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy',optimizer=adam_optimizer,metrics=['accuracy'])
history = model.fit(X_train, Y_train,epochs=5, batch_size=32, validation_data=(X_test, Y_test))



/var/folders/5k/gs40h3sn72lf36hj0p6k_9b80000gn/T/ipykernel_39587/1828367580.py:10: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False)


Epoch 1/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.1775 - loss: 2.2484 - val_accuracy: 0.2160 - val_loss: 2.1904
Epoch 2/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.2730 - loss: 2.1092 - val_accuracy: 0.2420 - val_loss: 2.0897
Epoch 3/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.3005 - loss: 1.9991 - val_accuracy: 0.2520 - val_loss: 2.0304
Epoch 4/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.3195 - loss: 1.9241 - val_accuracy: 0.2440 - val_loss: 1.9981
Epoch 5/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.3335 - loss: 1.8724 - val_accuracy: 0.2580 - val_loss: 1.9803


In [26]:
test_accuracy = model.evaluate(X_test, Y_test, batch_size=32)

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2580 - loss: 1.9803
